# folder data 處理

In [1]:
# import pandas as pd
# train_df = pd.read_csv('D:\\WorkSpace\\jupyter\\DataSet\\mango\\C1-P1_Train_Dev\\train.csv')
# dev_df = pd.read_csv('D:\\WorkSpace\\jupyter\\DataSet\\mango\\C1-P1_Train_Dev\\dev.csv')

# def train_img_path(id):
#     return 'D:\\WorkSpace\\jupyter\\DataSet\\mango\\C1-P1_Train_Dev\\C1-P1_Train\\%s' % id
# def get_cls_path(id, cls):
#     cls_path = 'D:\\WorkSpace\\jupyter\\DataSet\\mango\\C1-P1_Train_Dev\\C1-P1_Train\\%s\\%s'%(cls,id) 
#     image_path = 'D:\\WorkSpace\\jupyter\\DataSet\\mango\\C1-P1_Train_Dev\\C1-P1_Train\\' + id
#     return cls_path
      
# train_df['image_path'] = train_df['image_id'].apply(train_img_path)
# train_df['cls_path'] = train_df.apply(lambda x : get_cls_path(x['image_id'],x['label']),axis=1)

# def train_img_path(id):
#     return 'D:\\WorkSpace\\jupyter\\DataSet\\mango\\C1-P1_Train_Dev\\C1-P1_Dev\\%s' % id
# def get_cls_path(id, cls):
#     cls_path = 'D:\\WorkSpace\\jupyter\\DataSet\\mango\\C1-P1_Train_Dev\\C1-P1_Dev\\%s\\%s'%(cls,id) 
#     image_path = 'D:\\WorkSpace\\jupyter\\DataSet\\mango\\C1-P1_Train_Dev\\C1-P1_Dev\\' + id
#     return cls_path
      
# dev_df['image_path'] = dev_df['image_id'].apply(train_img_path)
# dev_df['cls_path'] = dev_df.apply(lambda x : get_cls_path(x['image_id'],x['label']),axis=1)

In [2]:
import shutil
from tqdm import tqdm

# with tqdm(total=len(train_df)) as pbar:
#     for i in range(len(train_df)):
#         series = train_df.iloc[i]
#         source, dest = series['image_path'], series['cls_path']
#         shutil.move(source,dest)
#         pbar.update(1)
        
# with tqdm(total=len(dev_df)) as pbar:
#     for i in range(len(dev_df)):
#         series = dev_df.iloc[i]
#         source, dest = series['image_path'], series['cls_path']
#         shutil.move(source,dest)
#         pbar.update(1)

# pytorch ImageFolder Dataset 

In [3]:
import time
import numpy as np
from torchvision.datasets import ImageFolder
from torchvision import transforms, models
import torch.nn as nn
from torch.autograd import Variable
import torch
import torch.optim as optim

#加上transforms
transform=transforms.Compose([
    transforms.Resize((512,512), interpolation=3), #缩放图片，保持长宽比不变，最短边的长为224像素,
#     transforms.CenterCrop(512), #从中间切出 224*224的图片    
#     transforms.RandomResizedCrop(512),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), #将图片转换为Tensor,归一化至[0,1]
    transforms.Normalize([0.5], [0.5])
])

train_dataset = ImageFolder('D:\\WorkSpace\\jupyter\\DataSet\\mango\\C1-P1_Train_Dev\\C1-P1_Train',transform=transform)
test_dataset = ImageFolder('D:\\WorkSpace\\jupyter\\DataSet\\mango\\C1-P1_Train_Dev\\C1-P1_Dev',transform=transform)

D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [4]:
train_dataset.classes

['A', 'B', 'C']

In [5]:
#输出第0张图片的大小
print(train_dataset[0][0].size)

<built-in method size of Tensor object at 0x000002048F697A20>


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# print(torch.cuda.get_device_name(device))

cuda:0


# 創建數據加載器

In [7]:
from torch.utils.data import DataLoader

torch.manual_seed(1)
BATCH_SIZE = 4

train_loader = DataLoader(
    dataset=train_dataset,# TensorDataset类型数据集
    batch_size=BATCH_SIZE,# mini batch size
    shuffle=True,# 设置随机洗牌
    num_workers=5# 加载数据的进程个数
)

test_loader = DataLoader(
    dataset=test_dataset,# TensorDataset类型数据集
    batch_size=BATCH_SIZE,# mini batch size
    shuffle=True,# 设置随机洗牌
    num_workers=5# 加载数据的进程个数
)
num_classes = 2

In [8]:
batch = next(iter(train_loader))

In [9]:
print(batch[0].size())
print(batch[1].size())

torch.Size([4, 3, 512, 512])
torch.Size([4])


# 構建網絡結構

In [10]:
# 两层卷积
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 使用序列工具快速构建
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(128 * 128 * 32, 2)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = out.view(out.size(0), -1)  # reshape
        out = self.fc(out)
        return out

In [11]:
# 3x3 卷积
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                     stride=stride, padding=1, bias=False)


# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


# ResNet Module
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=2):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 2)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(128 * 128, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [12]:
# 加載預訓練模型
model_ft = models.resnet34(pretrained=True)
num_ftrs = model_ft.fc.in_features

# 把resnet的全连接层fc 替换成自己设置的线性层nn.Linear
# 输入维度是resnet.fc.in_features, 输出是196维
model_ft.fc = nn.Linear(num_ftrs, num_classes)

In [13]:
from torchvision import models
# cnn = CNN()
# cnn = ResNet(ResidualBlock, [2, 2, 2, 2])
cnn = model_ft
print(cnn)
if torch.cuda.is_available():
    cnn = cnn.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

# 選擇損失函數

In [14]:
num_epochs = 10
LEARNING_RATE = 0.01

# 选择损失函数和优化方法
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(cnn.parameters(), lr=LEARNING_RATE, momentum=0.9)

# 将数据处理成Variable, 如果有GPU, 可以转成cuda形式
def get_variable(x):
    x = Variable(x)
    return x.cuda() if torch.cuda.is_available() else x

# 評估模型

In [15]:
def eval_model(model):
    correct = 0.0
    total = 0.0
    eval_loss = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            images = get_variable(images)
            labels = get_variable(labels)

            outputs = model(images)
            eval_loss += loss_func(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = 100.0 * correct / total
    eval_loss = eval_loss / total
#     print('Accuracy of the network on the test images: %d %%' % (
#         test_acc))
    return test_acc, eval_loss

# 訓練

In [16]:
losses = []
accuracies = []
test_accuracies = []
# set the model to train mode initially
cnn.train()

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)


for epoch in range(num_epochs):
    since = time.time()
    running_loss = 0.0
    running_correct = 0.0
    for step, (images, labels) in enumerate(train_loader):
        images = get_variable(images)
        labels = get_variable(labels)

        cnn.train()
        outputs = cnn(images)
        loss = loss_func(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # calculate the loss/acc later
        running_loss += loss.item()
        running_correct += (labels==predicted).sum().item()

    epoch_duration = time.time()-since
    epoch_loss = running_loss/len(train_loader)
    epoch_acc = 100/BATCH_SIZE*running_correct/len(train_loader)
    print("Epoch %s, duration: %d s, loss: %.4f, acc: %.4f" % (epoch+1, epoch_duration, epoch_loss, epoch_acc))
    
    losses.append(epoch_loss)
    accuracies.append(epoch_acc)

    # switch the model to eval mode to evaluate on test data
    cnn.eval()
    test_acc, test_loss = eval_model(cnn)
    print("eval_loss: %.4f, eval_acc: %.4f" % (test_loss, test_acc))
    test_accuracies.append(test_acc)
    
     # re-set the model to train mode after validating
    cnn.train()
    scheduler.step(test_acc)
    since = time.time()
    
print('Finished Training')


Epoch 1, duration: 1163 s, loss: 1.2102, acc: 41.1786
eval_loss: 0.2372, eval_acc: 51.2500
Epoch 2, duration: 1314 s, loss: 1.0006, acc: 50.0000
eval_loss: 0.6837, eval_acc: 45.0000
Epoch 3, duration: 1396 s, loss: 0.9617, acc: 52.5536
eval_loss: 0.2246, eval_acc: 54.8750
Epoch 4, duration: 1437 s, loss: 0.9229, acc: 55.3214
eval_loss: 0.2635, eval_acc: 57.8750
Epoch 5, duration: 1326 s, loss: 0.8976, acc: 57.3036
eval_loss: 0.1900, eval_acc: 64.7500
Epoch 6, duration: 1309 s, loss: 0.7562, acc: 64.6607
eval_loss: 0.1582, eval_acc: 72.0000
Epoch 7, duration: 1245 s, loss: 0.6681, acc: 69.8036
eval_loss: 0.1462, eval_acc: 73.7500
Epoch 8, duration: 1262 s, loss: 0.6252, acc: 72.2857
eval_loss: 0.1556, eval_acc: 71.6250
Epoch 9, duration: 1333 s, loss: 0.5904, acc: 74.0893
eval_loss: 0.1363, eval_acc: 77.3750
Epoch 10, duration: 1271 s, loss: 0.5347, acc: 76.8036
eval_loss: 0.1266, eval_acc: 78.0000
Finished Training


In [17]:
# Resize((512,512)
# Epoch 10, duration: 1372 s, loss: 0.4933, acc: 77.9643
# eval_loss: 0.1290, eval_acc: 76.2500

# 保存和加載整個網絡

In [18]:
# 保存和加载整个模型, 包括: 网络结构, 模型参数等
torch.save(cnn, 'model.pkl')
cnn = torch.load('model.pkl')

# 保存和加載網絡中的參數
torch.save(cnn.state_dict(), 'params.pkl')
cnn.load_state_dict(torch.load('params.pkl'))

